## Create data from seperated csv files to json file to import in mongo db.

### Import all the seperated csv file

In [1]:
import json  
import pandas as pd
import numpy as np

userData = pd.DataFrame(pd.read_csv('./users.csv'))
profileData = pd.DataFrame(pd.read_csv('./profile.csv'))
subscriptionData = pd.DataFrame(pd.read_csv('./subscription.csv'))
customerserviceData = pd.DataFrame(pd.read_csv('./customer_service.csv'))
profileHistoryData = pd.DataFrame(pd.read_csv('./profile_history.csv'))
technicalTeamData = pd.DataFrame(pd.read_csv('./technical_team.csv'))
movieIDData = pd.DataFrame(pd.read_csv('./movie_id.csv'))


In [2]:
userData.head(1)

,user_id,email,first,password,created_at,updated_at,subscription_id
0,1,at.nisi@google.ca,Shoshana Saunders,FLM59KPB4MF,07/01/2020,10/05/2020,3001


In [15]:
profileData.head(1)
profileData.describe()

,profile_id,user_id
count,400.000000,400.000000
mean,599.500000,151.745000
std,115.614301,87.907504
min,400.000000,1.000000
25%,499.750000,73.750000
50%,599.500000,151.000000
75%,699.250000,229.250000
max,799.000000,300.000000


In [16]:
subscriptionData.head(1)

,subscription_id,updated_at,price
0,3001,10/05/2020,19.95


In [17]:
customerserviceData.head(1)

,c_service_no,problem_desc,status,emp_id,created_date,updated_at,user_id
0,5001,Video Quality,Active,1025,20/08/2021,06/11/2021,134


In [18]:
profileHistoryData.head(1)
profileHistoryData.describe()

,movie_no,watched_movie_till,profile_id
count,500.000000,500.000000,500.000000
mean,495.506000,34.264000,601.464000
std,293.514844,19.323071,114.553978
min,4.000000,0.000000,401.000000
25%,234.000000,17.000000,499.750000
50%,489.000000,35.000000,607.500000
75%,743.250000,51.000000,698.250000
max,999.000000,66.000000,798.000000


In [19]:
technicalTeamData.head(1)


,emp_id,fname,lname
0,1001,Noah,Lawrence


In [20]:
movieIDData.head(1)

,_id
0,6264ba6667b279a3f7783e34


### Create Subscription data from csv

In [35]:

subscription = subscriptionData.to_numpy()
number_of_rows = subscription.shape[0]
def create_subscription():
    r,c = np.random.choice(number_of_rows, size=2)
    subs = {};
    subs['valid_till'] = datetime.strptime(subscription[r][1], '%d/%m/%Y')
    subs['price'] = 19.95
    return subs;
    

### Add Customer service data

In [36]:
cust_ser_data = customerserviceData.to_numpy()
technical_team = technicalTeamData.to_numpy()
def create_cust_service_data():
    arr = []
    no_of_data = random.randint(1, 3)
    
    cust_serv_data_rows = cust_ser_data.shape[0]
    technical_team_data_rows = technical_team.shape[0]
    for i in range(0, no_of_data+1):
        obj = {};
        r,c = np.random.choice(cust_serv_data_rows, size=2)
        r1,c1 = np.random.choice(technical_team_data_rows, size=2)
        obj['problem_desc'] = cust_ser_data[r][1];
        obj['status'] = cust_ser_data[r][2]
        obj['handled_by'] = { 'fname': technical_team[r1][1], 'lname': technical_team[r1][2] }
        obj['created_at'] = datetime.strptime(cust_ser_data[r1][4], '%d/%m/%Y')
        obj['updated_at'] = datetime.strptime(cust_ser_data[r1][5], '%d/%m/%Y')
        arr.append(obj)
        return arr   


### Create Profile data with hostory from csv file

In [37]:
import random 
from bson.objectid import ObjectId

movieId = movieIDData.to_numpy()
profiles = profileData.to_numpy()
profileHistory = profileHistoryData.to_numpy()
jsonData = [];

def create_profiles():
    arr = []
    no_of_profiles = random.randint(1, 5)
    profiles_number_of_rows = profiles.shape[0]
    history_number_of_rows = profileHistory.shape[0]
    for i in range(0, no_of_profiles):
        obj = {
        };
        obj['history'] = [];
        r,c = np.random.choice(profiles_number_of_rows, size=2)
        obj['name'] = profiles[r][2]
        no_of_history = random.randint(5, 30)
        for j in range(0, no_of_history):
            his = {}
            r1, c1 = np.random.choice(history_number_of_rows, size=2)
            his['watched_movie_no'] = int(profileHistory[r1][0])
            his['watched_movie_till'] = int(profileHistory[r1][1])
            obj['history'].append(his)
        arr.append(obj)
    return arr

In [38]:
from datetime import date, datetime
for ind, user in userData.iterrows():
    obj = {}

    obj['name'] = user['first']
    obj['email'] = user.email
    obj['password'] = user.password
    obj['created_at'] = datetime.strptime(user.created_at, '%d/%m/%Y')
    obj['updated_at'] = datetime.strptime(user.updated_at, '%d/%m/%Y')
    
    profile_arr = create_profiles()
    subscriptionObj = create_subscription()
    customer_service = create_cust_service_data()
    
    obj['billing_info'] = []
    obj['subscription'] = subscriptionObj
    obj['customer_service'] = customer_service
    obj['profiles'] = profile_arr
    jsonData.append(obj)    

def default(o):
    if isinstance(o, (datetime)):
        return o.isoformat()
    if isinstance(o, ObjectId):
         return str(o)
    
with open('user.json', 'w', encoding='UTF8', newline='') as f:
    json.dump(jsonData, f, ensure_ascii=False, indent=4, default=default)

In [33]:
movieData = pd.DataFrame(pd.read_csv('./movie.csv'))
movieData.head()

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59.0
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40.0


### Add Movie data

In [34]:
mjsonData = []

for i, movie in movieData.iterrows():
    obj = {}
    obj["_id"] = movie['Rank']
    obj["title"] = movie['Title']
    obj["genres"] = movie['Genre'].split(",")
    obj["description"] = movie['Description']
    obj["director"] = movie['Director']
    obj["actors"] = movie['Actors'].split(",")
    obj["year"] = movie['Year']
    obj["runtime"] = movie['Runtime (Minutes)']
    obj["rating"] = movie['Rating']
    obj["votes"] = movie['Votes']
    mjsonData.append(obj)
    
with open('movie.json', 'w', encoding='UTF8', newline='') as f:
    json.dump(mjsonData, f, ensure_ascii=False, indent=4)